# Notebook 02 — Entrenamiento Multi‑Label + MLflow + Interfaz (Gradio)
Este cuaderno cumple la **Parte 2** de la práctica:
- Transfer Learning + fine‑tuning
- Registro versionado en **MLflow** (params, métricas y modelo)
- Predicción con nuevos ejemplos
- Aprendizaje continuo: guardar feedback multi‑label y **retrain** rápido

**Etiquetas:** manzana, naranja, platano.



> Nota: archivos **.keras** son binarios (no texto).  
> Si los abres desde Jupyter (editor), aparecerá “not UTF‑8 encoded”.  
> Para *usarlos*, debes cargarlos con `tf.keras.models.load_model(...)` o descargarlos.


In [ ]:
if "FAST_MODE" not in globals():
    FAST_MODE = True


In [ ]:
# ==========================
# MODO RÁPIDO (evita demoras)
# ==========================
FAST_MODE = True   # <- Cambia a False para entrenamiento completo

# Tamaños y límites para acelerar
IMG_SIZE = (224, 224)

# Epochs por modo
EPOCHS_HEAD, EPOCHS_FT = ((2, 2) if FAST_MODE else (6, 6))

# Fine-tuning más liviano en modo rápido
UNFREEZE_LAST = (20 if FAST_MODE else 60)

# Batch más alto ayuda (si tu PC aguanta RAM/GPU)
BATCH = (32 if FAST_MODE else 16)

print("FAST_MODE:", FAST_MODE)
print("IMG_SIZE:", IMG_SIZE)
print("EPOCHS_HEAD, EPOCHS_FT:", EPOCHS_HEAD, EPOCHS_FT)
print("UNFREEZE_LAST:", UNFREEZE_LAST)
print("BATCH:", BATCH)


In [11]:
# =========================
# 1) Imports + checks
# =========================
import json, time, shutil
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

import tensorflow as tf
import mlflow

print("TensorFlow:", tf.__version__)
print("MLflow:", mlflow.__version__)

# Gradio (si no está instalado, descomenta la línea de pip)
try:
    import gradio as gr
    print("Gradio:", gr.__version__)
except Exception as e:
    print(" Gradio no está disponible aún. Ejecuta: !pip -q install gradio")

# =========================
# MLflow (Tracking + Model Registry)
# - Usamos FileStore local en Proyecto/mlruns
# - IMPORTANTE en Windows: usa URI file:///C:/...
# =========================
EXP_NAME = "frutas_multilabel"
MODEL_REG_NAME = "clasificador_frutas_multilabel"

TRACKING_DIR = Path("Proyecto/mlruns").resolve()
TRACKING_URI = TRACKING_DIR.as_uri()  # -> file:///C:/.../Proyecto/mlruns
# =========================
# Reparación (si se dañó mlruns)
# - A veces se queda una carpeta de experimento sin meta.yaml
#   y MLflow muestra: "Malformed experiment ... meta.yaml does not exist"
# - Este bloque crea meta.yaml faltantes SIN borrar tus runs.
# =========================
def repair_mlruns(mlruns_dir: Path):
    mlruns_dir = Path(mlruns_dir)
    mlruns_dir.mkdir(parents=True, exist_ok=True)
    now_ms = int(time.time() * 1000)

    # cada experimento es una carpeta numérica (excepto ".trash")
    for exp_dir in mlruns_dir.iterdir():
        if not exp_dir.is_dir():
            continue
        if exp_dir.name.startswith("."):
            continue
        if not exp_dir.name.isdigit():
            continue

        meta = exp_dir / "meta.yaml"
        if meta.exists():
            continue

        # Crear meta.yaml mínimo compatible
        # artifact_location: por defecto dentro del mismo mlruns (file://...)
        artifact_location = (mlruns_dir / exp_dir.name).as_uri()

        content = (
            f"artifact_location: {artifact_location}\n"
            f"creation_time: {now_ms}\n"
            f"experiment_id: '{exp_dir.name}'\n"
            f"last_update_time: {now_ms}\n"
            f"lifecycle_stage: active\n"
            f"name: recovered_{exp_dir.name}\n"
            f"tags: {{}}\n"
        )
        meta.write_text(content, encoding="utf-8")
        print(f" Reparado meta.yaml en experimento: {exp_dir.name}")

# Ejecutar reparación si hace falta
repair_mlruns(TRACKING_DIR)

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_registry_uri(TRACKING_URI)  # para que "Models" funcione en local
mlflow.set_experiment(EXP_NAME)

print("Tracking DIR:", TRACKING_DIR)
print(" Tracking URI:", TRACKING_URI)
print(" Experiment:", EXP_NAME)
print(" Model Registry Name:", MODEL_REG_NAME)


TensorFlow: 2.20.0
MLflow: 3.7.0
Gradio: 6.5.1
Tracking DIR: C:\Users\ASUS\practica6\Proyecto\mlruns
 Tracking URI: file:///C:/Users/ASUS/practica6/Proyecto/mlruns
 Experiment: frutas_multilabel
 Model Registry Name: clasificador_frutas_multilabel


In [12]:
# =========================
# 2) Cargar índice del dataset (creado en Notebook 01)
# =========================
DATA_INDEX = Path("Proyecto/dataset_index.csv")
assert DATA_INDEX.exists(), " Falta Proyecto/dataset_index.csv. Corre el Notebook 01 (EDA) primero."

df = pd.read_csv(DATA_INDEX)
assert {"path","label"}.issubset(df.columns), " El índice debe tener columnas: path,label"

class_names = ["manzana","naranja","platano"]
class_to_idx = {c:i for i,c in enumerate(class_names)}

# Validar que existan esas clases en el índice
found = sorted(df["label"].unique().tolist())
expected = sorted(class_names)
assert sorted([c for c in found if c in class_names]) == expected, f" Labels en índice: {found}"

print(" Filas en índice:", len(df))
print(" class_names:", class_names)

# Guardar class_names para que la UI use el mismo orden SIEMPRE
Path("class_names.json").write_text(json.dumps(class_names, indent=2), encoding="utf-8")
print(" Guardado class_names.json")


 Filas en índice: 104
 class_names: ['manzana', 'naranja', 'platano']
 Guardado class_names.json


In [13]:
# =========================
# 3) Estructura para MLflow + modelos + aprendizaje continuo
# =========================
(Path("Proyecto/mlruns")).mkdir(parents=True, exist_ok=True)
(Path("Proyecto/models")).mkdir(parents=True, exist_ok=True)

CONT_IMG_DIR = Path("Proyecto/continuous_data/images")
CONT_IMG_DIR.mkdir(parents=True, exist_ok=True)

LABELS_CSV = Path("Proyecto/continuous_data/labels.csv")
if not LABELS_CSV.exists():
    pd.DataFrame(columns=["filename"] + class_names).to_csv(LABELS_CSV, index=False)

print(" MLruns:", Path("Proyecto/mlruns").resolve())
print(" Models:", Path("Proyecto/models").resolve())
print(" Feedback images:", CONT_IMG_DIR.resolve())
print(" Feedback labels:", LABELS_CSV.resolve())


 MLruns: C:\Users\ASUS\practica6\Proyecto\mlruns
 Models: C:\Users\ASUS\practica6\Proyecto\models
 Feedback images: C:\Users\ASUS\practica6\Proyecto\continuous_data\images
 Feedback labels: C:\Users\ASUS\practica6\Proyecto\continuous_data\labels.csv


In [14]:
# =========================
# 4) Dataset Multi‑Label (mezcla rápida)
#    - Usa tus imágenes single-label como multi-label con 1 etiqueta activa
#    - Genera collages sintéticos con 2–3 frutas para aprender combinaciones
# =========================
IMG_SIZE = (224, 224)
BATCH = 32 if FAST_MODE else 16
SEED = 123

FAST_MODE = True  # ✅ True = más rápido (para pruebas), False = entrenamiento completo

# paths por clase
class_to_paths = {c: df[df["label"]==c]["path"].tolist() for c in class_names}
for c in class_names:
    assert len(class_to_paths[c]) > 0, f" Clase vacía en índice: {c}"

def onehot_multi(labels):
    y = np.zeros((len(class_names),), dtype=np.float32)
    for lab in labels:
        y[class_to_idx[lab]] = 1.0
    return y

def _load_img(path):
    img = Image.open(path).convert("RGB").resize(IMG_SIZE)
    return np.array(img, dtype=np.float32)

def sample_real_single(i):
    rng = np.random.default_rng(SEED + int(i))
    cls = rng.choice(class_names)
    path = rng.choice(class_to_paths[cls])
    x = _load_img(path)
    y = onehot_multi([cls])
    return x, y

def synth_collage(i):
    rng = np.random.default_rng(SEED*7 + int(i))
    # 1–3 clases presentes
    k = int(rng.choice([1,2,3], p=[0.45,0.40,0.15]))
    chosen = rng.choice(class_names, size=k, replace=False).tolist()

    bg = tuple(int(x) for x in rng.integers(210, 256, size=3))
    canvas = Image.new("RGB", (IMG_SIZE[1], IMG_SIZE[0]), bg)

    for cls in chosen:
        p = rng.choice(class_to_paths[cls])
        img = Image.open(p).convert("RGB")

        # Resize + rotación suave
        min_side = int(rng.integers(120, 180))
        img.thumbnail((min_side, min_side))
        img = img.rotate(float(rng.uniform(-18, 18)), expand=True)

        # Posición aleatoria
        max_x = max(1, canvas.size[0] - img.size[0])
        max_y = max(1, canvas.size[1] - img.size[1])
        x0 = int(rng.integers(0, max_x))
        y0 = int(rng.integers(0, max_y))
        canvas.paste(img, (x0, y0))

    x = np.array(canvas, dtype=np.float32)
    y = onehot_multi(chosen)
    return x, y

def make_mixed_dataset(n_samples, batch_size=16, shuffle=True, seed=123):
    ds = tf.data.Dataset.range(n_samples)

    def _map(i):
        def pyfn(j):
            j = int(j)
            # 60% collage, 40% real single-label
            if (j % 10) < 6:
                return synth_collage(j)
            else:
                return sample_real_single(j)
        x, y = tf.py_function(pyfn, [i], [tf.float32, tf.float32])
        x.set_shape((IMG_SIZE[0], IMG_SIZE[1], 3))
        y.set_shape((len(class_names),))
        return x, y

    ds = ds.map(_map, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(min(n_samples, 1000), seed=seed, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# Tamaños (ajusta si quieres más calidad vs velocidad)
N_TRAIN, N_VAL = (300, 60) if FAST_MODE else (800, 200)
train_ds = make_mixed_dataset(N_TRAIN, batch_size=BATCH, shuffle=True, seed=SEED)
val_ds   = make_mixed_dataset(N_VAL, batch_size=BATCH, shuffle=False, seed=SEED+999)

print(" train/val multi-label listos")


 train/val multi-label listos


In [15]:
# =========================
# 5) Modelo Multi‑Label (Transfer Learning)
#    - MobileNetV2 + salida sigmoid (3)
# =========================
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

augment = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.12),
    tf.keras.layers.RandomZoom(0.18),
    tf.keras.layers.RandomTranslation(0.10, 0.10),
    tf.keras.layers.RandomContrast(0.15),
], name="augment")

base = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)
base.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = augment(inputs)
x = preprocess_input(x)  # preprocess dentro del modelo
x = base(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.35)(x)
outputs = tf.keras.layers.Dense(len(class_names), activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="frutas_multilabel_mnv2")

model.summary()


Model: "frutas_multilabel_mnv2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ augment (Sequential)                 │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ true_divide (TrueDivide)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ subtract (Subtract)                  │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │           3,843 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,261,827 (8.63 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [16]:
# =========================
# 6) Entrenamiento (2 etapas) + MLflow (versionado)
# =========================
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_registry_uri(TRACKING_URI)
mlflow.set_experiment(EXP_NAME)
loss_fn = tf.keras.losses.BinaryCrossentropy()
def get_base_model(m):
    # intenta localizar el backbone dentro del modelo
    # (MobileNetV2 suele quedar como un sub-modelo)
    for layer in m.layers:
        if isinstance(layer, tf.keras.Model):
            return layer
    # fallback: busca por nombre típico
    for name in ["mobilenetv2_1.00_224", "MobileNetV2", "base"]:
        try:
            return m.get_layer(name)
        except Exception:
            pass
    return None

# Si el usuario ejecuta este bloque sin correr la celda de construcción,
# intentamos recuperar "base" desde el modelo ya cargado.
if "base" not in globals():
    try:
        base = get_base_model(model)
    except Exception:
        base = None

# Métricas (compatibles). AUC multi_label puede fallar en versiones viejas; hacemos fallback.
metrics = [tf.keras.metrics.BinaryAccuracy(threshold=0.5, name="bin_acc")]
try:
    metrics.append(tf.keras.metrics.AUC(curve="PR", multi_label=True, num_labels=len(class_names), name="auc_pr"))
    metrics.append(tf.keras.metrics.AUC(curve="ROC", multi_label=True, num_labels=len(class_names), name="auc_roc"))
    monitor_metric = "val_auc_pr"
    monitor_mode = "max"
except Exception as e:
    print(" AUC multi_label no disponible, usaré solo bin_acc.")
    monitor_metric = "val_bin_acc"
    monitor_mode = "max"

EPOCHS_HEAD, EPOCHS_FT = ((2, 2) if FAST_MODE else (6, 6))
LR_HEAD, LR_FT = 1e-3, 1e-5
UNFREEZE_LAST = (20 if FAST_MODE else 60)

cbs = [
    tf.keras.callbacks.EarlyStopping(monitor=monitor_metric, mode=monitor_mode, patience=(2 if FAST_MODE else 3), restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6),
]

def eval_multilabel_at_thr(model, ds, thr=0.5):
    y_true, y_pred = [], []
    for xb, yb in ds:
        p = model.predict(xb, verbose=0)
        y_true.append(yb.numpy())
        y_pred.append((p >= thr).astype(np.int32))
    y_true = np.vstack(y_true).astype(np.int32)
    y_pred = np.vstack(y_pred).astype(np.int32)

    eps = 1e-9
    out = {}
    for i, c in enumerate(class_names):
        tp = int(((y_true[:,i]==1) & (y_pred[:,i]==1)).sum())
        fp = int(((y_true[:,i]==0) & (y_pred[:,i]==1)).sum())
        fn = int(((y_true[:,i]==1) & (y_pred[:,i]==0)).sum())
        prec = tp / (tp + fp + eps)
        rec  = tp / (tp + fn + eps)
        f1   = 2*prec*rec / (prec + rec + eps)
        out[f"prec_{c}"] = float(prec)
        out[f"rec_{c}"]  = float(rec)
        out[f"f1_{c}"]   = float(f1)
    return out

with mlflow.start_run(run_name="train_multilabel_mnv2") as run:
    mlflow.log_params({
        "img_size": IMG_SIZE,
        "batch": BATCH,
        "n_train": N_TRAIN,
        "n_val": N_VAL,
        "epochs_head": EPOCHS_HEAD,
        "epochs_ft": EPOCHS_FT,
        "lr_head": LR_HEAD,
        "lr_ft": LR_FT,
        "unfreeze_last": UNFREEZE_LAST,
        "loss": "BinaryCrossentropy",
        "output": "sigmoid",
        "monitor": monitor_metric,
        "classes": ",".join(class_names),
    })
    mlflow.log_artifact("class_names.json")

    # Etapa 1: head
    base.trainable = False
    model.compile(optimizer=tf.keras.optimizers.Adam(LR_HEAD), loss=loss_fn, metrics=metrics)
    h1 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_HEAD, callbacks=cbs, verbose=1)

    # Etapa 2: fine-tuning
    base.trainable = True
    for layer in base.layers[:-UNFREEZE_LAST]:
        layer.trainable = False

    model.compile(optimizer=tf.keras.optimizers.Adam(LR_FT), loss=loss_fn, metrics=metrics)
    h2 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FT, callbacks=cbs, verbose=1)

    # Evaluación final
    eval_vals = model.evaluate(val_ds, verbose=0)
    final_metrics = {f"final_{k}": float(v) for k,v in zip(model.metrics_names, eval_vals)}
    mlflow.log_metrics(final_metrics)

    # Métricas a umbral fijo (thr=0.5)
    thr_metrics = eval_multilabel_at_thr(model, val_ds, thr=0.5)
    mlflow.log_metrics({f"thr05_{k}": v for k,v in thr_metrics.items()})

    # Guardar history
    hist = {"head": h1.history, "finetune": h2.history}
    Path("training_history_multilabel.json").write_text(json.dumps(hist, indent=2), encoding="utf-8")
    mlflow.log_artifact("training_history_multilabel.json")

    # Guardar modelo
    out_model = Path("Proyecto/models/best_model_multilabel.keras")
    model.save(out_model)
    mlflow.log_artifact(str(out_model))

    # Log model MLflow (best effort)
    try:
        import mlflow.tensorflow as mltf
        mltf.log_model(model, artifact_path="model", registered_model_name=MODEL_REG_NAME)
    except Exception:
        try:
            import mlflow.keras as mlk
            mlk.log_model(model, artifact_path="model", registered_model_name=MODEL_REG_NAME)
        except Exception as e:
            print(" No pude registrar el modelo con mlflow.*.log_model, pero el .keras sí quedó guardado.")

print(" Entrenamiento listo. Modelo:", Path("Proyecto/models/best_model_multilabel.keras"))


2026/02/06 14:18:11 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Epoch 1/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - auc_pr: 0.5364 - auc_roc: 0.5553 - bin_acc: 0.5289 - loss: 0.7799 - val_auc_pr: 0.7366 - val_auc_roc: 0.7513 - val_bin_acc: 0.7278 - val_loss: 0.5880 - learning_rate: 0.0010
Epoch 2/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - auc_pr: 0.7638 - auc_roc: 0.7728 - bin_acc: 0.7144 - loss: 0.5729 - val_auc_pr: 0.8264 - val_auc_roc: 0.8288 - val_bin_acc: 0.7833 - val_loss: 0.5017 - learning_rate: 0.0010
Epoch 1/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - auc_pr: 0.8134 - auc_roc: 0.8089 - bin_acc: 0.7278 - loss: 0.5601 - val_auc_pr: 0.8434 - val_auc_roc: 0.8417 - val_bin_acc: 0.8000 - val_loss: 0.4862 - learning_rate: 1.0000e-05
Epoch 2/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - auc_pr: 0.8138 - auc_roc: 0.8273 - bin_acc: 0.7389 - loss: 0.5378 - val_auc_pr: 0.8619 - val_auc_roc: 0.8518 - val_bin_acc: 0.8111 - val_loss: 0.4743 - learning_rate: 1.0000e-05


2026/02/06 14:24:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/06 14:24:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


 Entrenamiento listo. Modelo: Proyecto\models\best_model_multilabel.keras


C:\Users\ASUS\anaconda3\Lib\site-packages\mlflow\tracking\_model_registry\utils.py:215: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri)
Registered model 'clasificador_frutas_multilabel' already exists. Creating a new version of this model...
Created version '2' of model 'clasificador_frutas_multilabel'.


In [17]:
# =========================
# 7) Predicción Multi‑Label (función)
# =========================
def predict_multilabel_from_path(path, thr=0.5):
    img = Image.open(path).convert("RGB").resize(IMG_SIZE)
    x = np.array(img, dtype=np.float32)[None, ...]  # preprocess ya está dentro del modelo
    probs = model.predict(x, verbose=0)[0]
    present = [class_names[i] for i,p in enumerate(probs) if p >= thr]
    table = {class_names[i]: float(probs[i]) for i in range(len(class_names))}
    return present, table

# Ejemplo:
# present, table = predict_multilabel_from_path("ruta/imagen.jpg", thr=0.5)
# print(present, table)


In [18]:
# =========================
# 8) Guardar feedback multi‑label (para aprendizaje continuo)
#    - Guarda imagen en Proyecto/continuous_data/images
#    - Registra etiquetas reales (multi-hot) en labels.csv
# =========================
def save_feedback(image_path, labels_present):
    image_path = Path(image_path)
    assert image_path.exists(), " No existe la imagen a guardar."

    ts = int(time.time())
    fname = f"{ts}_{image_path.name}".replace(" ", "_")
    dst = CONT_IMG_DIR / fname
    shutil.copy(image_path, dst)

    row = {"filename": fname}
    for c in class_names:
        row[c] = 1 if c in labels_present else 0

    df_lab = pd.read_csv(LABELS_CSV)
    df_lab = pd.concat([df_lab, pd.DataFrame([row])], ignore_index=True)
    df_lab.to_csv(LABELS_CSV, index=False)

    return str(dst), row

print(" Feedback listo")


 Feedback listo


In [ ]:
# =========================
# 8.5) Guardado del retrain en Proyecto/models (para la interfaz)
# =========================
from pathlib import Path
import json
import mlflow
import tensorflow as tf

PROJECT_DIR = Path("Proyecto")
MODELS_DIR = PROJECT_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

def persist_retrain_to_models(model, class_names, thresholds_dict=None, tag="retrain"):
    """
    Guarda lo necesario para que tu UI cargue el último modelo reentrenado.
    - Proyecto/models/model.keras   (modelo vigente)
    - Proyecto/models/class_names.json
    - Proyecto/models/thresholds.json
    Además crea un snapshot en Proyecto/models/retrain_runs/<tag_timestamp>/.
    """
    if thresholds_dict is None:
        thresholds_dict = {c: 0.5 for c in class_names}

    # Archivos "vigentes" para la app
    (MODELS_DIR / "class_names.json").write_text(
        json.dumps({"classes": list(class_names)}, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    (MODELS_DIR / "thresholds.json").write_text(
        json.dumps({"thresholds": thresholds_dict}, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    model_path = MODELS_DIR / "model.keras"
    model.save(model_path)

    # Snapshot histórico
    snap_root = MODELS_DIR / "retrain_runs"
    snap_root.mkdir(parents=True, exist_ok=True)
    stamp = int(tf.timestamp().numpy())
    snap_dir = snap_root / f"{tag}_{stamp}"
    snap_dir.mkdir(parents=True, exist_ok=True)
    model.save(snap_dir / "model.keras")
    (snap_dir / "class_names.json").write_text((MODELS_DIR / "class_names.json").read_text(encoding="utf-8"), encoding="utf-8")
    (snap_dir / "thresholds.json").write_text((MODELS_DIR / "thresholds.json").read_text(encoding="utf-8"), encoding="utf-8")

    print(" Modelo vigente:", model_path)
    print(" Snapshot:", snap_dir)

def log_model_for_registry(model, artifact_path="model"):
    """
    Loguea el modelo en formato 'MLflow Model' (crea carpeta con MLmodel),
    requisito para poder hacer mlflow.register_model(...).
    """
    try:
        import mlflow.keras
        mlflow.keras.log_model(model, artifact_path=artifact_path)
        return artifact_path
    except Exception:
        import mlflow.tensorflow
        mlflow.tensorflow.log_model(model, artifact_path=artifact_path)
        return artifact_path


In [ ]:
from pathlib import Path
import json, time
import numpy as np
import tensorflow as tf
import mlflow

PROJECT_DIR = Path("Proyecto")
MODELS_DIR = PROJECT_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

def persist_retrain_to_models(model, class_names, thresholds_dict=None):
    """Guarda el modelo vigente para la interfaz + snapshot histórico."""
    model_path = MODELS_DIR / "model.keras"
    model.save(model_path)

    classes_path = MODELS_DIR / "class_names.json"
    classes_path.write_text(
        json.dumps({"classes": list(class_names)}, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    if thresholds_dict is None:
        thresholds_dict = {c: 0.5 for c in class_names}

    thr_path = MODELS_DIR / "thresholds.json"
    thr_path.write_text(
        json.dumps({"thresholds": thresholds_dict}, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    snap_dir = MODELS_DIR / "retrain_runs"
    snap_dir.mkdir(parents=True, exist_ok=True)
    run_folder = snap_dir / f"feedback_retrain_{int(time.time())}"
    run_folder.mkdir(parents=True, exist_ok=True)

    model.save(run_folder / "model.keras")
    (run_folder / "class_names.json").write_text(classes_path.read_text(encoding="utf-8"), encoding="utf-8")
    (run_folder / "thresholds.json").write_text(thr_path.read_text(encoding="utf-8"), encoding="utf-8")

    return str(model_path), str(run_folder)

def build_multilabel_mnv2(img_size_int, n_classes, dropout=0.2):
    inputs = tf.keras.Input(shape=(img_size_int, img_size_int, 3))
    base = tf.keras.applications.MobileNetV2(include_top=False, weights="imagenet", input_tensor=inputs)
    x = base.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    outputs = tf.keras.layers.Dense(n_classes, activation="sigmoid")(x)
    model = tf.keras.Model(inputs, outputs, name="multilabel_mnv2")
    return model, base

def log_model_for_registry(model, artifact_path="model"):
    """Crea el MLflow Model (con MLmodel) necesario para register_model."""
    try:
        import mlflow.keras
        mlflow.keras.log_model(model, artifact_path=artifact_path)
        return artifact_path
    except Exception:
        import mlflow.tensorflow
        mlflow.tensorflow.log_model(model, artifact_path=artifact_path)
        return artifact_path


In [ ]:
def retrain_quick_multilabel(epochs=2, lr=1e-5, unfreeze_last=20, fast=True):
    """
    Retrain rápido para Gradio:
    - 2 etapas: head + finetune (muy corto)
    - guarda en Proyecto/models/
    - loguea a MLflow y devuelve run_id
    """
    # --- Normalizar IMG_SIZE ---
    if isinstance(IMG_SIZE, (tuple, list)):
        img_size_int = int(IMG_SIZE[0])
    else:
        img_size_int = int(IMG_SIZE)

    n_classes = len(class_names)

    # --- Construcción ---
    model, base = build_multilabel_mnv2(img_size_int, n_classes, dropout=0.2)

    loss_fn = tf.keras.losses.BinaryCrossentropy()
    cbs = [
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=2, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1, min_lr=1e-6),
    ]

    # epochs por etapa
    ep_head = max(1, epochs // 2)
    ep_ft   = max(1, epochs - ep_head)

    with mlflow.start_run(run_name="feedback_retrain") as run:
        mlflow.log_params({
            "img_size": img_size_int,
            "epochs_head": ep_head,
            "epochs_ft": ep_ft,
            "lr": lr,
            "unfreeze_last": unfreeze_last,
            "classes": ",".join(class_names),
        })

        # ---- Etapa 1: head ----
        base.trainable = False
        model.compile(
            optimizer=tf.keras.optimizers.Adam(1e-3 if fast else lr),
            loss=loss_fn,
            metrics=[tf.keras.metrics.BinaryAccuracy(name="bin_acc")]
        )
        h1 = model.fit(train_ds, validation_data=val_ds, epochs=ep_head, callbacks=cbs, verbose=0)

        # ---- Etapa 2: fine-tune ----
        base.trainable = True
        if unfreeze_last is not None and unfreeze_last > 0:
            for layer in base.layers[:-unfreeze_last]:
                layer.trainable = False

        model.compile(
            optimizer=tf.keras.optimizers.Adam(lr),
            loss=loss_fn,
            metrics=[tf.keras.metrics.BinaryAccuracy(name="bin_acc")]
        )
        h2 = model.fit(train_ds, validation_data=val_ds, epochs=ep_ft, callbacks=cbs, verbose=0)

        # history
        hist = {"head": h1.history, "finetune": h2.history}
        Path("training_history_multilabel.json").write_text(json.dumps(hist, indent=2), encoding="utf-8")
        mlflow.log_artifact("training_history_multilabel.json")

        # guardar para la UI + snapshot
        thr_dict = {c: 0.5 for c in class_names}
        model_path, snap_path = persist_retrain_to_models(model, class_names, thresholds_dict=thr_dict)
        mlflow.log_artifact(model_path)
        mlflow.log_artifact(snap_path)

        # log como MLflow Model (para registry)
        model_art_path = log_model_for_registry(model, artifact_path="model")

        run_id = run.info.run_id

    return run_id


In [ ]:
# =========================
# 10) Interfaz (Gradio): predicción multi‑label + feedback + retrain
# =========================
import gradio as gr

TMP_DIR = Path("Proyecto/tmp_uploads")
TMP_DIR.mkdir(parents=True, exist_ok=True)

def gr_predict(img_np, thr, state_path):
    if img_np is None:
        return [], {}, "Sube una imagen", state_path

    ts = int(time.time())
    tmp = TMP_DIR / f"{ts}.png"
    Image.fromarray(img_np.astype("uint8")).save(tmp)

    present, table = predict_multilabel_from_path(tmp, thr=float(thr))
    msg = " Detectadas: " + (", ".join(present) if len(present)>0 else "Ninguna")
    return present, table, msg, str(tmp)

def gr_save(true_labels, state_path):
    if not state_path:
        return " Primero predice una imagen"
    dst, row = save_feedback(state_path, true_labels)
    return f" Feedback guardado: {dst}\n{row}"

def gr_retrain():
    run_id = retrain_quick_multilabel(epochs=3, lr=1e-5)
    return f" Retrain OK | run_id={run_id}"

with gr.Blocks() as demo:
    gr.Markdown("## Multi‑label Frutas (manzana/naranja/platano) — MLflow + Aprendizaje continuo")

    state_path = gr.State("")

    with gr.Row():
        img_in = gr.Image(type="numpy", label="Sube imagen")
        with gr.Column():
            thr = gr.Slider(0.1, 0.9, value=0.5, step=0.05, label="Umbral (thr)")
            pred_labels = gr.CheckboxGroup(choices=class_names, label="Etiquetas detectadas")
            probs_out = gr.JSON(label="Probabilidades (sigmoid)")
            msg_out = gr.Textbox(label="Mensaje")

    btn_pred = gr.Button(" Predecir")
    btn_pred.click(gr_predict, inputs=[img_in, thr, state_path], outputs=[pred_labels, probs_out, msg_out, state_path])

    gr.Markdown("### Feedback (etiquetas reales)")
    true_labels = gr.CheckboxGroup(choices=class_names, label="Marca todas las frutas que aparecen en la imagen")
    btn_save = gr.Button(" Guardar feedback")
    save_msg = gr.Textbox(label="Estado feedback")
    btn_save.click(gr_save, inputs=[true_labels, state_path], outputs=[save_msg])

    gr.Markdown("### Retrain rápido (nuevo run en MLflow)")
    btn_re = gr.Button(" Retrain")
    retrain_msg = gr.Textbox(label="Retrain status")
    btn_re.click(gr_retrain, inputs=[], outputs=[retrain_msg])

demo.launch(share=False, debug=True)


In [ ]:
import gradio as gr
from pathlib import Path
import json, time
import numpy as np
import tensorflow as tf
import mlflow

# ==========================
# CONFIG RÁPIDA
# ==========================
FAST_MODE = True  # cambia a False si quieres retrain más fuerte
RETRAIN_EPOCHS = 3 if FAST_MODE else 8
RETRAIN_LR = 1e-5
RETRAIN_UNFREEZE_LAST = 15 if FAST_MODE else 60

# Para retrain rápido (menos batches)
RETRAIN_TAKE_TRAIN = 60 if FAST_MODE else None
RETRAIN_TAKE_VAL = 15 if FAST_MODE else None

PROJECT_DIR = Path("Proyecto")
MODELS_DIR = PROJECT_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# HELPERS
# ==========================
def _img_size_int():
    if isinstance(IMG_SIZE, (tuple, list)):
        return int(IMG_SIZE[0])
    return int(IMG_SIZE)

def build_multilabel_mnv2(img_size_int, n_classes, dropout=0.2):
    inputs = tf.keras.Input(shape=(img_size_int, img_size_int, 3))
    base = tf.keras.applications.MobileNetV2(include_top=False, weights="imagenet", input_tensor=inputs)
    x = base.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    outputs = tf.keras.layers.Dense(n_classes, activation="sigmoid")(x)
    model = tf.keras.Model(inputs, outputs, name="multilabel_mnv2")
    return model, base

def persist_retrain_to_models(model, class_names, thresholds_dict=None):
    """Guarda modelo vigente para UI + snapshot histórico."""
    MODELS_DIR.mkdir(parents=True, exist_ok=True)

    # modelo "vigente"
    model_path = MODELS_DIR / "model.keras"
    model.save(model_path)

    # clases
    classes_path = MODELS_DIR / "class_names.json"
    classes_path.write_text(
        json.dumps({"classes": list(class_names)}, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    # thresholds
    if thresholds_dict is None:
        thresholds_dict = {c: 0.5 for c in class_names}

    thr_path = MODELS_DIR / "thresholds.json"
    thr_path.write_text(
        json.dumps({"thresholds": thresholds_dict}, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    # snapshot
    snap_dir = MODELS_DIR / "retrain_runs"
    snap_dir.mkdir(parents=True, exist_ok=True)
    run_folder = snap_dir / f"feedback_retrain_{int(time.time())}"
    run_folder.mkdir(parents=True, exist_ok=True)

    model.save(run_folder / "model.keras")
    (run_folder / "class_names.json").write_text(classes_path.read_text(encoding="utf-8"), encoding="utf-8")
    (run_folder / "thresholds.json").write_text(thr_path.read_text(encoding="utf-8"), encoding="utf-8")

    return str(model_path), str(run_folder)

def log_model_for_registry(model, artifact_path="model"):
    """Crea MLflow Model (MLmodel) -> necesario para register_model."""
    try:
        import mlflow.keras
        mlflow.keras.log_model(model, artifact_path=artifact_path)
        return artifact_path
    except Exception:
        import mlflow.tensorflow
        mlflow.tensorflow.log_model(model, artifact_path=artifact_path)
        return artifact_path

def retrain_quick_multilabel(epochs=3, lr=1e-5, unfreeze_last=20):
    """Retrain corto para Gradio, guarda en models/ y devuelve run_id."""
    img_size_int = _img_size_int()
    n_classes = len(class_names)

    # dataset reducido (para que no se demore)
    ds_train = train_ds
    ds_val = val_ds
    if RETRAIN_TAKE_TRAIN is not None:
        ds_train = ds_train.take(RETRAIN_TAKE_TRAIN)
    if RETRAIN_TAKE_VAL is not None:
        ds_val = ds_val.take(RETRAIN_TAKE_VAL)

    model, base = build_multilabel_mnv2(img_size_int, n_classes, dropout=0.2)
    loss_fn = tf.keras.losses.BinaryCrossentropy()

    cbs = [
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=2, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1, min_lr=1e-6),
    ]

    ep_head = max(1, epochs // 2)
    ep_ft   = max(1, epochs - ep_head)

    with mlflow.start_run(run_name="feedback_retrain") as run:
        mlflow.log_params({
            "img_size": img_size_int,
            "epochs_head": ep_head,
            "epochs_ft": ep_ft,
            "lr": lr,
            "unfreeze_last": unfreeze_last,
            "fast_mode": FAST_MODE,
            "classes": ",".join(class_names),
        })

        # ---- Etapa 1
        base.trainable = False
        model.compile(
            optimizer=tf.keras.optimizers.Adam(1e-3 if FAST_MODE else lr),
            loss=loss_fn,
            metrics=[tf.keras.metrics.BinaryAccuracy(name="bin_acc")]
        )
        h1 = model.fit(ds_train, validation_data=ds_val, epochs=ep_head, callbacks=cbs, verbose=0)

        # ---- Etapa 2
        base.trainable = True
        if unfreeze_last is not None and unfreeze_last > 0:
            for layer in base.layers[:-unfreeze_last]:
                layer.trainable = False

        model.compile(
            optimizer=tf.keras.optimizers.Adam(lr),
            loss=loss_fn,
            metrics=[tf.keras.metrics.BinaryAccuracy(name="bin_acc")]
        )
        h2 = model.fit(ds_train, validation_data=ds_val, epochs=ep_ft, callbacks=cbs, verbose=0)

        # history
        hist = {"head": h1.history, "finetune": h2.history}
        Path("training_history_multilabel.json").write_text(json.dumps(hist, indent=2), encoding="utf-8")
        mlflow.log_artifact("training_history_multilabel.json")

        # guardar en Proyecto/models
        thr_dict = {c: 0.5 for c in class_names}
        model_path, snap_path = persist_retrain_to_models(model, class_names, thresholds_dict=thr_dict)
        mlflow.log_artifact(model_path)

        # log como MLflow Model (para registry)
        model_art_path = log_model_for_registry(model, artifact_path="model")
        run_id = run.info.run_id

    return run_id

# ==========================
# CALLBACK GRADIO (COMPLETO)
# ==========================
def gr_retrain():
    try:
        run_id = retrain_quick_multilabel(
            epochs=RETRAIN_EPOCHS,
            lr=RETRAIN_LR,
            unfreeze_last=RETRAIN_UNFREEZE_LAST
        )
        return (
            "✅ Retrain listo\n"
            f"• run_id: {run_id}\n"
            "• Guardado (vigente): Proyecto/models/model.keras\n"
            "• Snapshot: Proyecto/models/retrain_runs/\n"
        )
    except Exception as e:
        return f"❌ Error en retrain: {type(e).__name__}: {e}"

# ==========================
# UI GRADIO (MINI)
# ==========================
with gr.Blocks(title="Entrenamiento Multi-Label (Frutas)") as demo:
    gr.Markdown("## Retrain rápido (feedback) → guarda en `Proyecto/models/`")
    btn = gr.Button("🔁 Retrain (rápido)")
    out = gr.Textbox(label="Estado", lines=6)
    btn.click(fn=gr_retrain, inputs=None, outputs=out)

demo.queue().launch()


## 11) Cómo ver MLflow (Experiments + Models)

**Importante:** los archivos `.keras` / `.h5` son binarios. Si los abres haciendo click en Jupyter, saldrá
"not UTF-8 encoded". Eso es normal.

**Para ver el registro en la web (MLflow UI):**
1. Abre PowerShell / Terminal en la carpeta raíz (donde está `Proyecto/`).
2. Ejecuta:
```
mlflow ui --backend-store-uri file:Proyecto/mlruns --host 127.0.0.1 --port 5000
```
3. Abre en el navegador: `http://127.0.0.1:5000`

En **Experiments** verás los runs y métricas. En **Models** verás el *Model Registry* (versiones) cuando el modelo se registre.


In [ ]:
# =========================
# 12) (Opcional) Registrar un modelo ya entrenado en Model Registry
#     - Útil si entrenaste antes SIN registered_model_name
#     - Esto hará que aparezca en la pestaña "Models"
# =========================
import mlflow
from mlflow.tracking import MlflowClient
from pathlib import Path

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_registry_uri(TRACKING_URI)
mlflow.set_experiment(EXP_NAME)

client = MlflowClient(tracking_uri=TRACKING_URI, registry_uri=TRACKING_URI)
exp = mlflow.get_experiment_by_name(EXP_NAME)
assert exp is not None, f"No existe el experimento {EXP_NAME}. Ejecuta el entrenamiento primero."

# Busca el run más reciente
runs = client.search_runs([exp.experiment_id], order_by=["attributes.start_time DESC"], max_results=5)
assert len(runs) > 0, "No hay runs. Entrena primero (celda 6 o 9)."

print("Runs recientes:")
for r in runs:
    print(" -", r.info.run_id, "|", r.info.run_name, "| metrics:", r.data.metrics)

latest_run_id = runs[0].info.run_id
model_uri = f"runs:/{latest_run_id}/model"

# Registrar (crea versión nueva en el mismo nombre)
print("\nRegistrando en Model Registry...")
mv = mlflow.register_model(model_uri=model_uri, name=MODEL_REG_NAME)
print(" Registrado:", mv.name, "| versión:", mv.version, "| run_id:", latest_run_id)

# Mostrar modelos registrados
print("\nModelos registrados:")
for m in client.search_registered_models():
    print(" -", m.name, "| versions:", [v.version for v in m.latest_versions])


In [ ]:
# =========================
# 13) Cargar el modelo desde MLflow (artefacto o registry)
# =========================
import tensorflow as tf
import mlflow.tensorflow
from mlflow.tracking import MlflowClient

client = MlflowClient(tracking_uri=TRACKING_URI, registry_uri=TRACKING_URI)

# Opción A: cargar la ÚLTIMA versión del registry
# (si ya registraste con la celda 6/9 o con la celda 12)
models = client.search_registered_models(filter_string=f"name='{MODEL_REG_NAME}'")
if len(models) > 0 and len(models[0].latest_versions) > 0:
    # toma la versión más alta
    versions = [int(v.version) for v in models[0].latest_versions]
    vmax = str(max(versions))
    model_from_registry = mlflow.tensorflow.load_model(f"models:/{MODEL_REG_NAME}/{vmax}")
    print(f" Modelo cargado desde registry: {MODEL_REG_NAME} v{vmax}")
else:
    model_from_registry = None
    print(" No hay modelos en registry todavía. Registra con la celda 12 o re-entrena con celda 6/9.")

# Opción B: cargar desde archivo .keras guardado
p = Path("Proyecto/models/best_model_multilabel_retrain.keras")
if p.exists():
    model_from_file = tf.keras.models.load_model(p)
    print(" Modelo cargado desde archivo:", p)
else:
    model_from_file = None
    print(" No existe el archivo:", p)



In [ ]:
import mlflow
import mlflow.tensorflow
from mlflow.tracking import MlflowClient
from pathlib import Path

# ====== AJUSTA ESTO (si quieres) ======
EXPERIMENT_NAME = "frutas_multilabel"
REGISTERED_MODEL_NAME = "clasificador_frutas_multilabel"
# ======================================

# 1) Asegura URIs correctas (Windows) para tracking + registry en el mismo folder
MLRUNS_DIR = Path("Proyecto/mlruns").resolve()
TRACKING_URI = MLRUNS_DIR.as_uri()   # file:///C:/.../Proyecto/mlruns

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_registry_uri(TRACKING_URI)

client = MlflowClient()

# 2) Busca el experimento (o créalo)
exp = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if exp is None:
    exp_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    exp_id = exp.experiment_id

# 3) Intenta registrar usando el "model" del ÚLTIMO RUN que lo tenga
runs = client.search_runs(
    experiment_ids=[exp_id],
    order_by=["attributes.start_time DESC"],
    max_results=50
)

run_id_with_model = None
for r in runs:
    arts = client.list_artifacts(r.info.run_id)
    art_names = {a.path for a in arts}
    if "model" in art_names:     # <- el artifact_path típico que aparece en tu columna "Models"
        run_id_with_model = r.info.run_id
        break

if run_id_with_model is None:
    # Si no encontró ningún run con "model", crea uno nuevo registrando el modelo en memoria (si existe)
    try:
        _ = model  # verifica si existe
        with mlflow.start_run(experiment_id=exp_id, run_name="register_model_from_memory") as run:
            mlflow.tensorflow.log_model(model, artifact_path="model", registered_model_name=REGISTERED_MODEL_NAME)
            print(" Registrado en Models:", REGISTERED_MODEL_NAME)
            print(" run_id:", run.info.run_id)
    except NameError:
        raise RuntimeError(
            "No encontré ningún RUN con artifact 'model' y tampoco existe la variable `model` en memoria.\n"
            "Solución: vuelve a ejecutar la celda de entrenamiento (la que crea `model`) y luego corre esta celda."
        )
else:
    # Registrar el modelo del run encontrado
    model_uri = f"runs:/{run_id_with_model}/model"
    mv = mlflow.register_model(model_uri=model_uri, name=REGISTERED_MODEL_NAME)

    print("Registrado en Models:", REGISTERED_MODEL_NAME)
    print("Versión creada:", mv.version)
    print(" Desde run_id:", run_id_with_model)
    print(" Tracking/Registry URI:", TRACKING_URI)

print("\n Ahora ve a MLflow → pestaña 'Models' y refresca (F5).")


In [4]:
# ==========================
# EXPORTAR (persistir) EL RETRAIN A /models
# ==========================
from pathlib import Path
import json
import numpy as np

MODELS_DIR = Path("models")
MODELS_DIR.mkdir(parents=True, exist_ok=True)

def persist_retrain_to_models(model, classes, thresholds, out_dir: Path = MODELS_DIR):
    """
    Guarda lo necesario para que tu app/UI cargue el último reentrenamiento:
    - modelo keras: models/model.keras
    - clases: models/classes.json
    - thresholds: models/thresholds.json (formato compatible con tu loader)
    """
    out_dir.mkdir(parents=True, exist_ok=True)

    # 1) Modelo
    model_path = out_dir / "model.keras"
    model.save(model_path)

    # 2) Clases
    (out_dir / "classes.json").write_text(
        json.dumps({"classes": list(classes)}, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    # 3) Thresholds (compatible con load_thresholds_or_default)
    thr = np.asarray(thresholds).astype(float).tolist()
    thr_dict = {c: float(t) for c, t in zip(classes, thr)}

    (out_dir / "thresholds.json").write_text(
        json.dumps({"thresholds": thr_dict}, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    print(" Retrain exportado a:", out_dir.resolve())
    print("   -", model_path.name)
    print("   - classes.json")
    print("   - thresholds.json")


In [1]:
cd RUTA_DE_TU_PROYECTO
git init
git add .
git commit -m "Primera version del proyecto"
git branch -M main
git remote add origin https://github.com/TU_USUARIO/TU_REPOSITORIO.git
git push -u origin main


SyntaxError: invalid syntax (7821147.py, line 1)